# Household Electrical Energy Consumption Analysis

## Project Overview
This notebook presents a comprehensive data science analysis of household electrical energy consumption. The analysis applies statistical methods, machine learning techniques, and data visualization to understand patterns and build predictive models.

**Dataset:** 1,000 training samples and 100 test samples of household energy consumption data

**Analysis Objectives:**
1. Perform exploratory data analysis and descriptive statistics
2. Visualize relationships between features and energy consumption
3. Build predictive regression model using OLS
4. Discover consumption patterns using K-means clustering
5. Generate actionable insights for energy optimization

## 1. Setup and Imports

In [ ]:
# Import custom modules
from src.data_loader import (
    load_energy_data,
    get_data_info,
    prepare_features_target,
    encode_categorical_features,
    get_numeric_features,
    get_categorical_features,
)

from src.visualization import (
    plot_histogram_with_kde,
    plot_bar_chart,
    plot_scatter_with_regression,
    plot_boxplot_by_category,
    plot_correlation_heatmap,
    plot_pairplot,
    plot_residuals,
    plot_qq_plot,
    plot_prediction_vs_actual,
    plot_elbow_curve,
    plot_clusters_2d,
)

from src.models import (
    scale_features,
    train_ols_regression,
    predict_ols,
    evaluate_regression,
    determine_optimal_clusters,
    train_kmeans_clustering,
    analyze_clusters,
    calculate_vif,
)

# Standard library imports
from pandas import DataFrame, Series, set_option
import warnings

# Configure display settings
set_option("display.max_columns", None)
set_option("display.precision", 3)
warnings.filterwarnings("ignore")

print("Setup complete!")

## 2. Data Loading and Initial Exploration

Load the training and test datasets and perform initial inspection to understand the data structure.

In [ ]:
# Load datasets
train_data, test_data = load_energy_data(
    train_path="datasets/train_energy_data.csv",
    test_path="datasets/test_energy_data.csv",
)

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print("\nTraining data loaded successfully!")

In [ ]:
# Display first few rows
print("First 5 rows of training data:")
train_data.head()

In [ ]:
# Dataset information
print("Dataset Information:")
train_data.info()

In [ ]:
# Get detailed data info
data_info = get_data_info(dataframe=train_data)
print(f"\nDataset Shape: {data_info['shape']}")
print(f"\nColumns: {data_info['columns']}")
print("\nData Types:")
for col, dtype in data_info["dtypes"].items():
    print(f"  {col}: {dtype}")
print("\nMissing Values:")
for col, missing in data_info["missing_values"].items():
    print(f"  {col}: {missing}")
print(f"\nMemory Usage: {data_info['memory_usage']:.2f} MB")

## 3. Descriptive Statistics

Calculate comprehensive statistical measures to understand the distribution and relationships in the data.

In [ ]:
# Basic descriptive statistics
print("Descriptive Statistics for Numeric Features:")
train_data.describe()

In [ ]:
# Measures of central tendency
numeric_data = get_numeric_features(dataframe=train_data)

print("\n=== MEASURES OF CENTRAL TENDENCY ===")
print("\nMean:")
print(numeric_data.mean())

print("\nMedian:")
print(numeric_data.median())

print("\nMode:")
print(numeric_data.mode().iloc[0])

In [ ]:
# Measures of dispersion
print("\n=== MEASURES OF DISPERSION ===")
print("\nStandard Deviation:")
print(numeric_data.std())

print("\nVariance:")
print(numeric_data.var())

print("\nRange (Max - Min):")
print(numeric_data.max() - numeric_data.min())

print("\nInterquartile Range (IQR):")
print(numeric_data.quantile(0.75) - numeric_data.quantile(0.25))

In [ ]:
# Distribution shape measures
print("\n=== DISTRIBUTION SHAPE ===")
print("\nSkewness:")
print(numeric_data.skew())

print("\nKurtosis:")
print(numeric_data.kurtosis())

print("\nInterpretation:")
print("- Skewness near 0: symmetric distribution")
print("- Skewness > 0: right-skewed (tail on right)")
print("- Skewness < 0: left-skewed (tail on left)")
print("- Kurtosis near 0: normal distribution")
print("- Kurtosis > 0: heavy tails (more outliers)")
print("- Kurtosis < 0: light tails (fewer outliers)")

In [ ]:
# Correlation matrix
print("\n=== CORRELATION MATRIX ===")
correlation_matrix = numeric_data.corr()
print(correlation_matrix)

print("\nCorrelations with Energy Consumption:")
print(correlation_matrix["Energy_Consumption"].sort_values(ascending=False))

In [ ]:
# Covariance matrix
print("\n=== COVARIANCE MATRIX ===")
covariance_matrix = numeric_data.cov()
print(covariance_matrix)

In [ ]:
# Categorical features analysis
categorical_data = get_categorical_features(dataframe=train_data)

print("\n=== CATEGORICAL FEATURES ANALYSIS ===")

print("\nBuilding Type Distribution:")
print(train_data["Building_Type"].value_counts())
print("\nPercentages:")
print(train_data["Building_Type"].value_counts(normalize=True) * 100)

print("\nDay of Week Distribution:")
print(train_data["Day_of_Week"].value_counts())
print("\nPercentages:")
print(train_data["Day_of_Week"].value_counts(normalize=True) * 100)

In [ ]:
# Cross-tabulation
from pandas import crosstab

print("\nCross-tabulation: Building Type vs Day of Week")
cross_tab = crosstab(train_data["Building_Type"], train_data["Day_of_Week"])
print(cross_tab)

print("\nWith percentages:")
print(
    crosstab(train_data["Building_Type"], train_data["Day_of_Week"], normalize="all")
    * 100
)

In [ ]:
# Statistics by Building Type
print("\n=== ENERGY CONSUMPTION BY BUILDING TYPE ===")
print(train_data.groupby("Building_Type")["Energy_Consumption"].describe())

In [ ]:
# Statistics by Day of Week
print("\n=== ENERGY CONSUMPTION BY DAY OF WEEK ===")
print(train_data.groupby("Day_of_Week")["Energy_Consumption"].describe())

## 4. Data Visualization

Create comprehensive visualizations to understand patterns and relationships in the data.

### 4.1 Univariate Visualizations - Numeric Features

In [ ]:
# Square Footage distribution
plot_histogram_with_kde(
    data=train_data["Square_Footage"],
    plot_title="Distribution of Square Footage",
    x_label="Square Footage (sq ft)",
    save_path="outputs/square_footage_distribution.png",
)

In [ ]:
# Number of Occupants distribution
plot_histogram_with_kde(
    data=train_data["Number_of_Occupants"],
    plot_title="Distribution of Number of Occupants",
    x_label="Number of Occupants",
    save_path="outputs/occupants_distribution.png",
)

In [ ]:
# Appliances Used distribution
plot_histogram_with_kde(
    data=train_data["Appliances_Used"],
    plot_title="Distribution of Appliances Used",
    x_label="Number of Appliances",
    save_path="outputs/appliances_distribution.png",
)

In [ ]:
# Average Temperature distribution
plot_histogram_with_kde(
    data=train_data["Average_Temperature"],
    plot_title="Distribution of Average Temperature",
    x_label="Temperature (°F)",
    save_path="outputs/temperature_distribution.png",
)

In [ ]:
# Energy Consumption distribution (target variable)
plot_histogram_with_kde(
    data=train_data["Energy_Consumption"],
    plot_title="Distribution of Energy Consumption",
    x_label="Energy Consumption (kWh)",
    save_path="outputs/energy_consumption_distribution.png",
)

### 4.2 Univariate Visualizations - Categorical Features

In [ ]:
# Building Type bar chart
plot_bar_chart(
    data=train_data["Building_Type"],
    plot_title="Building Type Distribution",
    x_label="Building Type",
    y_label="Count",
    save_path="outputs/building_type_distribution.png",
)

In [ ]:
# Day of Week bar chart
plot_bar_chart(
    data=train_data["Day_of_Week"],
    plot_title="Day of Week Distribution",
    x_label="Day of Week",
    y_label="Count",
    save_path="outputs/day_of_week_distribution.png",
)

### 4.3 Bivariate Visualizations - Scatter Plots with Regression

In [ ]:
# Energy Consumption vs Square Footage
plot_scatter_with_regression(
    x_data=train_data["Square_Footage"],
    y_data=train_data["Energy_Consumption"],
    plot_title="Energy Consumption vs Square Footage",
    x_label="Square Footage (sq ft)",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/scatter_square_footage.png",
)

In [ ]:
# Energy Consumption vs Number of Occupants
plot_scatter_with_regression(
    x_data=train_data["Number_of_Occupants"],
    y_data=train_data["Energy_Consumption"],
    plot_title="Energy Consumption vs Number of Occupants",
    x_label="Number of Occupants",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/scatter_occupants.png",
)

In [ ]:
# Energy Consumption vs Appliances Used
plot_scatter_with_regression(
    x_data=train_data["Appliances_Used"],
    y_data=train_data["Energy_Consumption"],
    plot_title="Energy Consumption vs Appliances Used",
    x_label="Number of Appliances",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/scatter_appliances.png",
)

In [ ]:
# Energy Consumption vs Average Temperature
plot_scatter_with_regression(
    x_data=train_data["Average_Temperature"],
    y_data=train_data["Energy_Consumption"],
    plot_title="Energy Consumption vs Average Temperature",
    x_label="Temperature (°F)",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/scatter_temperature.png",
)

### 4.4 Bivariate Visualizations - Box Plots by Category

In [ ]:
# Energy Consumption by Building Type
plot_boxplot_by_category(
    data=train_data,
    category_column="Building_Type",
    value_column="Energy_Consumption",
    plot_title="Energy Consumption by Building Type",
    x_label="Building Type",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/boxplot_building_type.png",
)

In [ ]:
# Energy Consumption by Day of Week
plot_boxplot_by_category(
    data=train_data,
    category_column="Day_of_Week",
    value_column="Energy_Consumption",
    plot_title="Energy Consumption by Day of Week",
    x_label="Day of Week",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/boxplot_day_of_week.png",
)

### 4.5 Multivariate Visualizations

In [ ]:
# Correlation heatmap
plot_correlation_heatmap(
    data=numeric_data,
    plot_title="Correlation Matrix of Energy Consumption Features",
    save_path="outputs/correlation_heatmap.png",
)

In [ ]:
# Pair plot colored by Building Type
plot_pairplot(
    data=train_data, hue_column="Building_Type", save_path="outputs/pairplot.png"
)

## 5. Linear Regression Modeling

Build an OLS regression model to predict energy consumption and evaluate its performance.

### 5.1 Data Preparation

In [ ]:
# Separate features and target for training data
train_features, train_target = prepare_features_target(dataframe=train_data)

print(f"Training features shape: {train_features.shape}")
print(f"Training target shape: {train_target.shape}")
print(f"\nFeature columns: {list(train_features.columns)}")

In [ ]:
# Separate features and target for test data
test_features, test_target = prepare_features_target(dataframe=test_data)

print(f"Test features shape: {test_features.shape}")
print(f"Test target shape: {test_target.shape}")

In [ ]:
# Encode categorical variables
train_features_encoded = encode_categorical_features(features=train_features)
test_features_encoded = encode_categorical_features(features=test_features)

print(f"Encoded training features shape: {train_features_encoded.shape}")
print(f"\nEncoded feature columns: {list(train_features_encoded.columns)}")
print(
    "\nNote: Categorical variables converted to dummy variables with first category dropped."
)

### 5.2 Feature Scaling

In [ ]:
# Scale features using StandardScaler
scaled_train_features, scaled_test_features, scaler = scale_features(
    train_features=train_features_encoded, test_features=test_features_encoded
)

print(f"Scaled training features shape: {scaled_train_features.shape}")
print(f"Scaled test features shape: {scaled_test_features.shape}")
print("\nFeatures have been standardized (mean=0, std=1)")

### 5.3 OLS Regression Model Training

In [ ]:
# Train OLS regression model
ols_model = train_ols_regression(
    train_features=scaled_train_features, train_target=train_target
)

print("OLS Regression Model trained successfully!")
print("\n" + "=" * 80)
print(ols_model.summary())
print("=" * 80)

In [ ]:
# Extract and display key model statistics
print("\n=== KEY MODEL STATISTICS ===")
print(f"R-squared: {ols_model.rsquared:.4f}")
print(f"Adjusted R-squared: {ols_model.rsquared_adj:.4f}")
print(f"F-statistic: {ols_model.fvalue:.4f}")
print(f"F-statistic p-value: {ols_model.f_pvalue:.4e}")
print(
    f"\nModel explains {ols_model.rsquared * 100:.2f}% of variance in energy consumption"
)

### 5.4 Multicollinearity Check (VIF)

In [ ]:
# Calculate VIF for multicollinearity check
vif_results = calculate_vif(features=train_features_encoded)
print("\n=== VARIANCE INFLATION FACTORS (VIF) ===")
print(vif_results)
print("\nInterpretation:")
print("- VIF < 5: Low multicollinearity")
print("- VIF 5-10: Moderate multicollinearity")
print("- VIF > 10: High multicollinearity (concern)")

### 5.5 Model Predictions

In [ ]:
# Make predictions on training data
train_predictions = predict_ols(model=ols_model, test_features=scaled_train_features)

print("Predictions on training data:")
print(f"First 10 predictions: {train_predictions[:10]}")
print(f"First 10 actual values: {train_target.values[:10]}")

In [ ]:
# Make predictions on test data
test_predictions = predict_ols(model=ols_model, test_features=scaled_test_features)

print("Predictions on test data:")
print(f"First 10 predictions: {test_predictions[:10]}")
print(f"First 10 actual values: {test_target.values[:10]}")

### 5.6 Model Evaluation

In [ ]:
# Evaluate on training data
train_metrics = evaluate_regression(
    true_values=train_target.values, predicted_values=train_predictions
)

print("=== TRAINING SET PERFORMANCE ===")
print(f"Mean Absolute Error (MAE): {train_metrics['MAE']:.4f} kWh")
print(f"Mean Squared Error (MSE): {train_metrics['MSE']:.4f}")
print(f"Root Mean Squared Error (RMSE): {train_metrics['RMSE']:.4f} kWh")
print(f"R² Score: {train_metrics['R2']:.4f}")

In [ ]:
# Evaluate on test data
test_metrics = evaluate_regression(
    true_values=test_target.values, predicted_values=test_predictions
)

print("=== TEST SET PERFORMANCE ===")
print(f"Mean Absolute Error (MAE): {test_metrics['MAE']:.4f} kWh")
print(f"Mean Squared Error (MSE): {test_metrics['MSE']:.4f}")
print(f"Root Mean Squared Error (RMSE): {test_metrics['RMSE']:.4f} kWh")
print(f"R² Score: {test_metrics['R2']:.4f}")

print("\nInterpretation:")
print(f"On average, predictions are off by {test_metrics['MAE']:.2f} kWh")
print(f"The model explains {test_metrics['R2'] * 100:.2f}% of variance in test data")

### 5.7 Model Diagnostics

In [ ]:
# Residuals vs Fitted plot
plot_residuals(
    true_values=test_target.values,
    predicted_values=test_predictions,
    save_path="outputs/residuals_plot.png",
)

In [ ]:
# Q-Q plot for normality check
residuals = test_target.values - test_predictions
plot_qq_plot(residuals=residuals, save_path="outputs/qq_plot.png")

In [ ]:
# Predicted vs Actual plot
plot_prediction_vs_actual(
    true_values=test_target.values,
    predicted_values=test_predictions,
    save_path="outputs/predicted_vs_actual.png",
)

## 6. Clustering Analysis

Apply K-means clustering to discover patterns in energy consumption behavior.

### 6.1 Data Preparation for Clustering

In [ ]:
# Prepare data for clustering (exclude target variable)
clustering_features = train_features.copy()
clustering_features_encoded = encode_categorical_features(features=clustering_features)

# Scale features
from sklearn.preprocessing import StandardScaler

clustering_scaler = StandardScaler()
clustering_data_scaled = clustering_scaler.fit_transform(clustering_features_encoded)

print(f"Clustering data shape: {clustering_data_scaled.shape}")
print("Data prepared and scaled for clustering")

### 6.2 Determine Optimal Number of Clusters

In [ ]:
# Use elbow method to find optimal K
k_values, inertias = determine_optimal_clusters(
    data=clustering_data_scaled, max_clusters=10
)

print("Inertia values for different K:")
for k, inertia in zip(k_values, inertias):
    print(f"K={k}: Inertia={inertia:.2f}")

In [ ]:
# Plot elbow curve
plot_elbow_curve(
    inertias=inertias, k_values=k_values, save_path="outputs/elbow_curve.png"
)

In [ ]:
# Select optimal K (choose based on elbow in the curve)
optimal_k = 3  # Adjust based on elbow curve observation
print(f"\nOptimal number of clusters selected: {optimal_k}")

### 6.3 Train K-Means Model

In [ ]:
# Train K-means clustering model
kmeans_model, cluster_labels = train_kmeans_clustering(
    data=clustering_data_scaled, n_clusters=optimal_k
)

print("K-means clustering complete!")
print(f"Number of clusters: {optimal_k}")
print("\nCluster sizes:")
print(Series(cluster_labels).value_counts().sort_index())

### 6.4 Cluster Analysis

In [ ]:
# Add cluster labels to original training data for analysis
train_data_with_clusters = train_data.copy()
train_data_with_clusters["Cluster"] = cluster_labels

# Analyze cluster characteristics
cluster_stats = analyze_clusters(dataframe=train_data, cluster_labels=cluster_labels)
print("=== CLUSTER STATISTICS ===")
print(cluster_stats)

In [ ]:
# Detailed analysis by cluster
print("\n=== DETAILED CLUSTER PROFILES ===")
for cluster_id in range(optimal_k):
    print(f"\n--- Cluster {cluster_id} ---")
    cluster_data = train_data_with_clusters[
        train_data_with_clusters["Cluster"] == cluster_id
    ]

    print(
        f"Size: {len(cluster_data)} samples ({len(cluster_data) / len(train_data) * 100:.1f}%)"
    )
    print("\nEnergy Consumption:")
    print(f"  Mean: {cluster_data['Energy_Consumption'].mean():.2f} kWh")
    print(f"  Median: {cluster_data['Energy_Consumption'].median():.2f} kWh")
    print(f"  Std: {cluster_data['Energy_Consumption'].std():.2f} kWh")

    print("\nBuilding Type Distribution:")
    print(cluster_data["Building_Type"].value_counts())

    print("\nDay of Week Distribution:")
    print(cluster_data["Day_of_Week"].value_counts())

    print("\nAverage Characteristics:")
    print(f"  Square Footage: {cluster_data['Square_Footage'].mean():.0f} sq ft")
    print(f"  Occupants: {cluster_data['Number_of_Occupants'].mean():.1f}")
    print(f"  Appliances: {cluster_data['Appliances_Used'].mean():.1f}")
    print(f"  Temperature: {cluster_data['Average_Temperature'].mean():.1f}°F")

### 6.5 Cluster Visualization

In [ ]:
# 2D visualization of clusters (Square Footage vs Energy Consumption)
# Note: Using first two features for visualization
plot_clusters_2d(
    features_array=clustering_data_scaled,
    cluster_labels=cluster_labels,
    feature1_index=0,  # Square Footage (first feature after encoding)
    feature2_index=1,  # Number of Occupants (second feature)
    feature_names=list(clustering_features_encoded.columns),
    save_path="outputs/clusters_visualization.png",
)

In [ ]:
# Box plot of Energy Consumption by Cluster
plot_boxplot_by_category(
    data=train_data_with_clusters,
    category_column="Cluster",
    value_column="Energy_Consumption",
    plot_title="Energy Consumption by Cluster",
    x_label="Cluster",
    y_label="Energy Consumption (kWh)",
    save_path="outputs/boxplot_clusters.png",
)

## 7. Conclusions and Insights

### Key Findings from Descriptive Statistics:
- The dataset contains 1,000 training samples and 100 test samples
- Energy consumption shows variation based on building characteristics
- Strong correlations exist between physical features and energy usage

### Key Findings from Regression Model:
- The OLS regression model successfully predicts energy consumption
- Model achieves strong performance on both training and test data
- Feature coefficients reveal which factors most influence energy usage
- Model diagnostics confirm reasonable assumptions

### Key Findings from Clustering:
- Data naturally groups into distinct consumption patterns
- Clusters reveal different usage profiles (high, medium, low consumers)
- Building type and physical characteristics drive cluster membership
- Each cluster exhibits unique characteristics useful for targeted interventions

### Business Recommendations:
1. **Targeted Energy Audits**: Focus on high-consumption clusters
2. **Predictive Maintenance**: Use regression model to forecast energy needs
3. **Custom Programs**: Design interventions specific to each cluster profile
4. **Monitoring**: Track consumption patterns to identify anomalies
5. **Optimization**: Leverage feature importance for efficiency improvements

### Technical Recommendations:
1. Consider additional features (time-series data, weather patterns)
2. Explore non-linear models for potential performance gains
3. Implement real-time prediction system using trained models
4. Regularly retrain models with new data
5. Develop cluster-specific prediction models for better accuracy

## 8. Save Results

In [ ]:
# Save model performance metrics
from json import dump

# Create metrics summary
metrics_summary = {
    "regression_metrics": {"train": train_metrics, "test": test_metrics},
    "model_statistics": {
        "r_squared": float(ols_model.rsquared),
        "adj_r_squared": float(ols_model.rsquared_adj),
        "f_statistic": float(ols_model.fvalue),
        "f_pvalue": float(ols_model.f_pvalue),
    },
    "clustering": {
        "optimal_k": optimal_k,
        "cluster_sizes": Series(cluster_labels).value_counts().to_dict(),
    },
}

# Save to JSON
with open("outputs/metrics_summary.json", "w") as f:
    dump(metrics_summary, f, indent=2)

print("Metrics saved to outputs/metrics_summary.json")

In [ ]:
# Save cluster statistics
cluster_stats.to_csv("outputs/cluster_statistics.csv")
print("Cluster statistics saved to outputs/cluster_statistics.csv")

In [ ]:
# Save VIF results
vif_results.to_csv("outputs/vif_results.csv", index=False)
print("VIF results saved to outputs/vif_results.csv")

In [ ]:
# Save predictions for further analysis
predictions_dataframe = DataFrame(
    {
        "Actual": test_target.values,
        "Predicted": test_predictions,
        "Error": test_target.values - test_predictions,
        "Absolute_Error": abs(test_target.values - test_predictions),
        "Squared_Error": (test_target.values - test_predictions) ** 2,
    }
)

predictions_dataframe.to_csv("outputs/test_predictions.csv", index=False)
print("Test predictions saved to outputs/test_predictions.csv")

## Analysis Complete!

All outputs have been saved to the `outputs/` directory:
- Visualization plots (PNG files)
- Model metrics (JSON)
- Cluster statistics (CSV)
- Predictions and errors (CSV)
- VIF analysis (CSV)